For this project, you will be working with the Hop Teaming dataset, a dataset which aims to capture referrals between healthcare providers based on Medicare claims. 
We will be using multiple data sets HOP_teaming(hospitals referal), NEPPS(doctors NPI and specifics), Taxonomy(details about the taxonomy code in NPESS) and the ZIP_CBSA(locations and connecting to doctors)

In [1]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

In [3]:
#reading HOP.csv for overveiw and glance
chunks = pd.read_csv('data/DocGraph_Hop_Teaming_2018.csv',
           chunksize = 10000)

In [5]:
chunks

In [11]:
chunk = next(chunks)

In [12]:
chunk

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
20000,1508825647,1740277375,64,108,29.370,37.453
20001,1508055161,1740277391,17,31,26.323,58.271
20002,1508848748,1740277425,21,26,63.577,88.700
20003,1508818980,1740277466,11,13,48.615,35.208
20004,1508838095,1740277508,41,43,54.953,67.435
...,...,...,...,...,...,...
29995,1508251505,1740595198,11,13,25.846,30.692
29996,1497935837,1740595362,51,53,66.264,84.008
29997,1508806498,1740595362,21,21,90.762,91.419
29998,1508883133,1740595362,39,42,92.667,76.537


We want to eliminate "accidental" referrals, so filter the hop teaming data so that the transaction_count is at least 50 and the average_day_wait is less than 50.

In [13]:
chunk[(chunk['transaction_count'] >= 50) & (chunk['average_day_wait']<50)]

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
20000,1508825647,1740277375,64,108,29.370,37.453
20013,1508879818,1740277516,92,107,48.953,67.793
20016,1508892282,1740277516,87,91,43.330,69.353
20020,1508827379,1740277516,181,188,40.165,72.921
20028,1508815333,1740277623,172,258,34.880,57.357
...,...,...,...,...,...,...
29938,1497937536,1740592336,38,60,20.550,29.606
29973,1497933162,1740594217,49,51,3.980,17.500
29982,1508125386,1740594332,56,84,42.393,69.225
29994,1508014564,1740595198,42,149,0.188,1.188


In [14]:
#Reading the NEPSS_NPI for glance
chunks_npi = pd.read_csv('data/npidata_pfile_20050523-20230212.csv',
           chunksize = 10000)

In [38]:
chunk_npi = next(chunks_npi)
chunk_npi

C:\Users\upadh\AppData\Local\Temp\ipykernel_384\3734988901.py:1: DtypeWarning: Columns (17,24,32,39,40,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,89,90,143,147,151,154,157,159,161,165,166,167,169,170) have mixed types. Specify dtype option on import or set low_memory=False.
  chunk_npi = next(chunks_npi)


,NPI,Entity Type Code,Replacement NPI,Employer Identification Number (EIN),Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,...,Healthcare Provider Taxonomy Group_7,Healthcare Provider Taxonomy Group_8,Healthcare Provider Taxonomy Group_9,Healthcare Provider Taxonomy Group_10,Healthcare Provider Taxonomy Group_11,Healthcare Provider Taxonomy Group_12,Healthcare Provider Taxonomy Group_13,Healthcare Provider Taxonomy Group_14,Healthcare Provider Taxonomy Group_15,Certification Date
40000,1518944925,1,NaN,NaN,NaN,ROJO,RODOLFO,NaN,DR.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40001,1427035831,1,NaN,NaN,NaN,HELLING,ERIC,NaN,DR.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40002,1063499473,1,NaN,NaN,NaN,MIMS,STEVEN,C,MR.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40003,1972580389,1,NaN,NaN,NaN,ROSENBLOOM,TAMMIE,SINGER,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40004,1376520627,1,NaN,NaN,NaN,COLIHAN,LAURA,CHATHERINE,MS.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,1720050735,1,NaN,NaN,NaN,MACKEY,WENDY,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49996,1548232556,1,NaN,NaN,NaN,IKEDA,CLYDE,J,DR.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49997,1457323461,1,NaN,NaN,NaN,HALVORSEN,MARGARET,ROSE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49998,1275505281,1,NaN,NaN,NaN,LOYD,DEBORAH,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
#list(chunk_npi.columns)
#chunk_npi.shape
#chunk_npi[.info]

['NPI',
 'Entity Type Code',
 'Replacement NPI',
 'Employer Identification Number (EIN)',
 'Provider Organization Name (Legal Business Name)',
 'Provider Last Name (Legal Name)',
 'Provider First Name',
 'Provider Middle Name',
 'Provider Name Prefix Text',
 'Provider Name Suffix Text',
 'Provider Credential Text',
 'Provider Other Organization Name',
 'Provider Other Organization Name Type Code',
 'Provider Other Last Name',
 'Provider Other First Name',
 'Provider Other Middle Name',
 'Provider Other Name Prefix Text',
 'Provider Other Name Suffix Text',
 'Provider Other Credential Text',
 'Provider Other Last Name Type Code',
 'Provider First Line Business Mailing Address',
 'Provider Second Line Business Mailing Address',
 'Provider Business Mailing Address City Name',
 'Provider Business Mailing Address State Name',
 'Provider Business Mailing Address Postal Code',
 'Provider Business Mailing Address Country Code (If outside U.S.)',
 'Provider Business Mailing Address Telephone Nu

In [55]:
chunk_npi['Provider Organization Name (Legal Business Name)'].value_counts()
chunk_npi['Healthcare Provider Taxonomy Code_1'].value_counts()
chunk_npi[ 'Healthcare Provider Primary Taxonomy Switch_10'].value_counts()

X    2
Name: Healthcare Provider Primary Taxonomy Switch_10, dtype: int64

In [ ]:
Y    9431
X     518
N      51

In [56]:
#subsetting the df
npi = chunk_npi[['NPI',
 'Entity Type Code',
 'Provider Organization Name (Legal Business Name)',
 'Provider Last Name (Legal Name)',
 'Provider First Name',
 'Provider Middle Name',
 'Provider Name Prefix Text',
 'Provider Name Suffix Text',
 'Provider Credential Text',
 'Provider First Line Business Practice Location Address',
 'Provider Second Line Business Practice Location Address',
 'Provider Business Practice Location Address City Name',
 'Provider Business Practice Location Address State Name',
 'Provider Business Practice Location Address Postal Code',
           'Healthcare Provider Taxonomy Code_1',
           'Healthcare Provider Primary Taxonomy Switch_1',
           'Healthcare Provider Taxonomy Code_2',
           'Healthcare Provider Primary Taxonomy Switch_2', 
           'Healthcare Provider Taxonomy Code_3',
           'Healthcare Provider Primary Taxonomy Switch_3',     
           'Healthcare Provider Taxonomy Code_4',
           'Healthcare Provider Primary Taxonomy Switch_4',
           'Healthcare Provider Taxonomy Code_5',
           'Healthcare Provider Primary Taxonomy Switch_5',
            'Healthcare Provider Taxonomy Code_6',
           'Healthcare Provider Primary Taxonomy Switch_6',
        'Healthcare Provider Taxonomy Code_7',
           'Healthcare Provider Primary Taxonomy Switch_7',
                 'Healthcare Provider Taxonomy Code_8',
           'Healthcare Provider Primary Taxonomy Switch_8',
                 'Healthcare Provider Taxonomy Code_9',
           'Healthcare Provider Primary Taxonomy Switch_9',
                 'Healthcare Provider Taxonomy Code_10',
           'Healthcare Provider Primary Taxonomy Switch_10',
                 'Healthcare Provider Taxonomy Code_11',
           'Healthcare Provider Primary Taxonomy Switch_11',
                 'Healthcare Provider Taxonomy Code_12',
           'Healthcare Provider Primary Taxonomy Switch_12',
                 'Healthcare Provider Taxonomy Code_13',
           'Healthcare Provider Primary Taxonomy Switch_13',
                 'Healthcare Provider Taxonomy Code_14',
           'Healthcare Provider Primary Taxonomy Switch_14',
                 'Healthcare Provider Taxonomy Code_15',
           'Healthcare Provider Primary Taxonomy Switch_15'
            ]]

In [57]:
x = []

for i in range (1:16):
    if f'Healthcare Provider Primary Taxonomy Switch_{i}' == 'Y'
    x.append[;1:14]
    else 
    
#use the loop for selecting the rows with switch as 'Y' and collect all codes in code column to create tiny tables and then concat them

,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,...,Healthcare Provider Taxonomy Code_11,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Taxonomy Code_12,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Taxonomy Code_13,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Healthcare Provider Primary Taxonomy Switch_15
40000,1518944925,1,NaN,ROJO,RODOLFO,NaN,DR.,NaN,M.D.,12864 BISCAYNE BLVD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40001,1427035831,1,NaN,HELLING,ERIC,NaN,DR.,NaN,MD,3851 ROGER BROOKE DR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40002,1063499473,1,NaN,MIMS,STEVEN,C,MR.,NaN,CRNA,7152 COCA SABAL LN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40003,1972580389,1,NaN,ROSENBLOOM,TAMMIE,SINGER,NaN,NaN,LICSW,2550 UNIVERSITY AVE W,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40004,1376520627,1,NaN,COLIHAN,LAURA,CHATHERINE,MS.,NaN,RPA C,826 WASHINGTON ST,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,1720050735,1,NaN,MACKEY,WENDY,NaN,NaN,NaN,APRN,230 BOSTON POST RD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49996,1548232556,1,NaN,IKEDA,CLYDE,J,DR.,NaN,MD,1199 BUSH ST,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49997,1457323461,1,NaN,HALVORSEN,MARGARET,ROSE,NaN,NaN,WHNP,8450 SEASONS PKWY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49998,1275505281,1,NaN,LOYD,DEBORAH,NaN,NaN,NaN,LCSW,1443 CORPORATE WAY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
for column in (chunk_npi.columns):
    
    # Select column contents by column
    # name using [] operator
    columnSeriesObj = cunk[column]
    print('Column Name : ', column)
    print('Column Contents : ', columnSeriesObj.values)